In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [3]:
data_dir = os.path.join(Path.cwd().parent, 'data', 'raw_data', "personal_budget_dataset.csv")
df = pd.read_csv(data_dir)

In [5]:
df.head()
df.isna().sum()

,Month,Category,Budget,Actual Spending,Income,Spending Deviation
0,1,Groceries,402,322,4000,-80
1,1,Rent,1000,1002,4000,2
2,1,Utilities,201,222,4000,21
3,1,Entertainment,192,166,4000,-26
4,1,Transportation,114,101,4000,-13


In [7]:
df.groupby(by=['Month', 'Actual Spending']).sum()
df[(df["Category"] == "Transportation") & ( df["Month"] == 1 )]

Category  Budget  Income  Spending Deviation
Month Actual Spending                                                    
1     101              Transportation     114    4000                 -13
      120                  Healthcare     121    4000                  -1
      166               Entertainment     192    4000                 -26
      222                   Utilities     201    4000                  21
      272                  Dining Out     256    4000                  16
...                               ...     ...     ...                 ...
24    263                  Dining Out     173    4000                  90
      285               Entertainment     198    4000                  87
      425                   Groceries     345    4000                  80
      661                     Savings     601    4000                  60
      994                        Rent    1000    4000                  -6

[192 rows x 4 columns]

### Check Feature Importance

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


data = df
data['Category'] = LabelEncoder().fit_transform(data['Category'])

x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [15]:
param = {"max_depth": range(0,10),
         "max_features": range(0,6)}

model = GridSearchCV(RandomForestRegressor(), param_grid=param, cv=5)

In [19]:
model.fit(x_train, y_train)

c:\Users\batou\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
75 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\batou\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\batou\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\batou\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.p

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': range(0, 10),
                         'max_features': range(0, 6)})

In [20]:
model.best_params_

{'max_depth': 9, 'max_features': 5}

In [22]:
model.best_estimator_.feature_importances_

array([0.0897603 , 0.02866505, 0.33959477, 0.54197988, 0.        ])

In [25]:
y_pred = model.predict(x_test)
mse = mean_squared_error(y_pred, y_test)
mse

441.17739529622327

In [26]:
data.columns

Index(['Month', 'Category', 'Budget', 'Actual Spending', 'Income',
       'Spending Deviation'],
      dtype='object')